In [4]:
!pip install shap


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import os # Import the os module

# Load dataset
data = pd.read_csv('/content/dataset-uci.csv')

# Show basic info
print("Dataset Shape:", data.shape)
print("Columns:", data.columns)
print("Missing Values:\n", data.isnull().sum())
print("First 5 Rows:\n", data.head())

# ====== STEP 1: Data Preprocessing ======

# Fill missing values if any (here using median)
data = data.fillna(data.median(numeric_only=True))

# Define Features and Target
X = data.drop(['Gallstone Status'], axis=1)   # Assuming 'Outcome' is the target column
y = data['Gallstone Status']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# ====== STEP 2: Model Building ======

# Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# ====== STEP 3: Model Evaluation ======

# Accuracy, Precision, Recall, F1 Score
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC AUC Score:", roc_auc)

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Random model line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend()
plt.show()

# Create the 'models' directory if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

# Save model and scaler
joblib.dump(model, 'models/random_forest_model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')

NameError: name 'model' is not defined

In [ ]:
import shap

# Initialize TreeExplainer for RandomForest
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Summary Plot (Global Feature Importance)
# Access the SHAP values for the positive class (index 1) across all samples and features
shap.summary_plot(shap_values[:, :, 1], X_test, feature_names=X.columns)

# Force Plot (For one individual prediction)
shap.initjs()
# Access the expected value and SHAP values for the positive class for the first sample
shap.force_plot(explainer.expected_value[1], shap_values[0, :, 1], X_test[0], feature_names=X.columns)